# Functions


In [1]:
import dataclasses
from datetime import date


@dataclasses.dataclass
class RehearsalSlot:
    """Represents a single, structured rehearsal event."""
    rehearsal_date: date
    day_of_week: str  # Standardized, lowercase e.g., 'wednesday'
    start_time: int   # Military time, e.g., 1100
    end_time: int     # Military time, e.g., 1200


In [2]:
import dataclasses
from abc import ABC, abstractmethod
from typing import TypeAlias


TimeInterval: TypeAlias = tuple[int, int]

class UnavailabilityConstraint(ABC):
    """Abstract base class for all dancer unavailability constraints."""
    @abstractmethod
    def get_conflicting_intervals(self, slot: RehearsalSlot) -> list[TimeInterval]:
        pass

@dataclasses.dataclass(frozen=True)
class DayOfWeekConstraint(UnavailabilityConstraint):
    """Represents an unavailability for an entire day of the week."""
    day_of_week: str

    def get_conflicting_intervals(self, slot: RehearsalSlot) -> list[TimeInterval]:
        if self.day_of_week == slot.day_of_week:
            return [(slot.start_time, slot.end_time)]
        return []


@dataclasses.dataclass(frozen=True)
class TimeOnDayConstraint(UnavailabilityConstraint):
    """
    Represents an unavailability for a specific time interval on a given day of the week.
    e.g., "Mondays from 900 to 1200"
    """
    day_of_week: str  # e.g., 'monday'
    start_time: int   # Military time, e.g., 900
    end_time: int     # Military time, e.g., 1200

    def get_conflicting_intervals(self, slot: RehearsalSlot) -> list[TimeInterval]:
        """
        Checks for conflicts if the day matches AND the time intervals overlap.
        """
        if self.day_of_week != slot.day_of_week:
            return [] # No conflict if it's not on the right day.

        # Classic interval overlap check: max(starts) < min(ends)
        overlap_start = max(self.start_time, slot.start_time)
        overlap_end = min(self.end_time, slot.end_time)

        if overlap_start < overlap_end:
            # A conflict exists! Return the actual interval of the conflict.
            return [(overlap_start, overlap_end)]
        
        return []

In [3]:
def check_for_conflict(
    rehearsal: RehearsalSlot,
    dancer_constraints: list[UnavailabilityConstraint]
) -> list[TimeInterval]:
    """Checks for conflicts and returns a list of conflicting intervals."""
    all_conflicts = []
    for constraint in dancer_constraints:
        # print(f"constraint is {constraint}")
        conflicting_intervals = constraint.get_conflicting_intervals(rehearsal)
        all_conflicts.extend(conflicting_intervals)
    return all_conflicts

In [4]:
from lark import Lark
from lark.exceptions import LarkError
from typing import List, Optional, Tuple

# Assuming your SemanticValidationError is defined as before
class SemanticValidationError(ValueError):
    """Raised when the syntax is valid but the meaning is not."""
    pass

def validate_and_parse_all_constraints(parser: Lark, text: str) -> Tuple[bool, Optional[List], Optional[str]]:
    """
    Parses a comma-separated string of constraints, validating each one individually
    and collecting ALL errors found.

    Returns:
        A tuple of (success, results, error_message).
        - If ALL constraints are valid, (True, [list_of_constraints], None).
        - If ANY constraint is invalid, (False, None, "multi-line error report").
    """
    parsed_constraints = []
    error_messages = []

    constraint_chunks = [chunk.strip() for chunk in text.split(',') if chunk.strip()]
    
    if not constraint_chunks:
        return (True, [], None)

    for chunk in constraint_chunks:
        try:
            parsed_result = parser.parse(chunk)
            
            # THE FIX: Use .append() to add the single object returned by the parser.
            # .extend() would fail with a TypeError because the parsed_result is not a list.
            parsed_constraints.append(parsed_result)

        except LarkError:
            msg = f"- Constraint '{chunk}' is not grammatically valid."
            error_messages.append(msg)
        
        except SemanticValidationError as e:
            msg = f"- Constraint '{chunk}' is not logically valid: {e}"
            error_messages.append(msg)
            
        except Exception as e:
            # This block will now only catch truly unexpected errors.
            msg = f"- An unexpected error occurred on constraint '{chunk}': {e}"
            error_messages.append(msg)

    if error_messages:
        header = "Found one or more invalid constraints:"
        # Corrected usage of '\n' for newlines
        final_error_message = f"{header}\n" + "\n".join(error_messages)
        return (False, None, final_error_message)
    else:
        return (True, parsed_constraints, None)

## Grammar Specification

In [5]:
from lark import Lark

constraint_grammar = r"""
    ?start: conflict_text -> process_conflicts

    conflict_text: unavailability_spec ("," unavailability_spec)*

    // An unavailability can now be a simple day or a more complex time-on-day spec
    unavailability_spec: time_on_day_spec | day_spec

    // A simple day spec (e.g., "Monday") becomes a DayOfWeekConstraint
    day_spec: MONDAY | TUESDAY | WEDNESDAY | THURSDAY | FRIDAY | SATURDAY | SUNDAY

    // A time-on-day spec combines a day with a time range
    time_on_day_spec: day_spec time_range

    // --- Time Range Rules ---
    // These define the "until", "after", and "X-Y" formats
    ?time_range: until_range | after_range | explicit_range

    until_range: "until"i time 
               | "before"i time -> build_until_range
    after_range: "after"i time -> build_after_range
    explicit_range: time "-" time -> build_explicit_range

    // --- Time Parsing Rules ---
    // This rule is the key to flexible time parsing.
    // It captures an integer and optional AM/PM marker.
    time: INT -> number_only
        | INT AM_PM -> number_with_ampm

    AM_PM: "am"i | "pm"i

    // --- Day of Week Terminals (Unchanged) ---
    MONDAY:    "monday"i    | "mon"i   | "mo"i | "m"i
    TUESDAY:   "tuesday"i   | "tues"i  | "tu"i
    WEDNESDAY: "wednesday"i | "wed"i   | "we"i | "w"i
    THURSDAY:  "thursday"i  | "thurs"i | "th"i
    FRIDAY:    "friday"i    | "fri"i   | "fr"i | "f"i
    SATURDAY:  "saturday"i  | "sat"i   | "sa"i
    SUNDAY:    "sunday"i    | "sun"i   | "su"i

    %import common.INT
    %import common.WS
    %ignore WS
"""

In [6]:
from lark import Lark, Transformer, v_args
from typing import TypeAlias, Optional

# --- A custom exception for semantic errors ---
class SemanticValidationError(ValueError):
    """Raised when the syntax is valid but the meaning is not."""
    pass


@v_args(inline=True)
class ConstraintTransformer(Transformer):
    # --- Time Normalization and Validation Helper ---
    def _normalize_time(self, hour: int, am_pm: Optional[str] = None) -> int:
        """
        Converts various time formats to a military time integer.
        This is where semantic validation happens!
        """
        hour = int(hour)
        
        if am_pm:
            am_pm = am_pm.lower()
            if not (1 <= hour <= 12):
                raise ValueError(f"Invalid 12-hour format: Hour '{hour}' must be between 1 and 12.")
            if am_pm == 'pm' and hour < 12:
                hour += 12
            elif am_pm == 'am' and hour == 12: # Midnight case
                hour = 0
        else:
            # Handles military time (e.g., 1400) or simple hours (e.g., 2 in "2-4")
            if hour > 24: # This catches your "after 25" case!
                raise SemanticValidationError(f"Invalid 24-hour format: Hour '{hour}' cannot be greater than 24.")
            if hour >= 1 and hour <= 7: # Heuristic for "2-4" meaning PM
                # If you write "w 2-4", you almost certainly mean 2 PM to 4 PM.
                hour += 12
        
        return hour * 100 # Convert to military time format (e.g., 14 -> 1400)

    # --- Time Parsing Methods ---
    def number_only(self, hour_token):
        return self._normalize_time(hour_token.value)

    def number_with_ampm(self, hour_token, am_pm_token):
        return self._normalize_time(hour_token.value, am_pm_token.value)

    # --- Time Range Builders ---
    def build_until_range(self, end_time):
        return (0, end_time) # 0 is the start of the day
        
    def until_range(self, children):
        return self.build_until_range(children)
        
    def build_after_range(self, start_time):
        return (start_time, 2359) # 2359 is the end of the day

    def build_explicit_range(self, start_time, end_time):
        if start_time >= end_time:
            raise ValueError(f"Invalid time range: Start time {start_time} must be before end time {end_time}.")
        return (start_time, end_time)

    def time_specifier(self, children):
        return children[0]

    def time_on_day_constraint(self, children):
        # NOW, this method will correctly receive ['monday', (0, 1200)]
        # and the unpacking will succeed.
        day_of_week, time_tuple = children
        start_time, end_time = time_tuple
        
    def until_time(self, children):
        hour = int(children[0].value)
        # ... (your logic)
        return (0, hour * 100)
        
        # --- Constraint Object Creation ---
    def time_on_day_spec(self, day_of_week_obj, time_range_tuple):
        # day_of_week_obj is a DayOfWeekConstraint, we just need its string value
        day_str = day_of_week_obj.day_of_week
        start_time, end_time = time_range_tuple
        return TimeOnDayConstraint(day_str, start_time, end_time)

    # --- Day of Week Rules (now create DayOfWeekConstraint) ---
    def MONDAY(self, _): return DayOfWeekConstraint("monday")
    def TUESDAY(self, _): return DayOfWeekConstraint("tuesday")
    def WEDNESDAY(self, _): return DayOfWeekConstraint("wednesday")
    def THURSDAY(self, _): return DayOfWeekConstraint("thursday")
    def FRIDAY(self, _): return DayOfWeekConstraint("friday")
    def SATURDAY(self, _): return DayOfWeekConstraint("saturday")
    def SUNDAY(self, _): return DayOfWeekConstraint("sunday")

    # --- Structural/Collection Rules ---
    def day_spec(self, day): return day
    def unavailability_spec(self, spec): return spec
    
    @v_args(inline=False)
    def conflict_text(self, items): return list(items)
    
    @v_args(inline=True)
    def process_conflicts(self, conflicts_list): return conflicts_list

## executable code

In [7]:
# Step 3.1: "Compile" the grammar into a parser object
from lark import Lark

print("--- Compiling Grammar ---")
constraint_parser = Lark(constraint_grammar)
print("✅ Grammar compiled successfully.\n")

--- Compiling Grammar ---
✅ Grammar compiled successfully.



In [8]:
constraint_parser = Lark(constraint_grammar, start='start', parser='lalr', transformer=ConstraintTransformer())
dancer_unavailability_string = "mon, w, friday, m"
parsed_constraints = constraint_parser.parse(dancer_unavailability_string)

# print(f"Input string: '{dancer_unavailability_string}'")
# print("Parsed constraint objects:")
# for c in parsed_constraints:
#     print(f"  - {c}")
# print(f"Type of parsed_constraints: {type(parsed_constraints)}\n")

In [9]:
rehearsal_1 = RehearsalSlot(date(2024, 9, 4), 'wednesday', 1800, 1930) # Should conflict
rehearsal_2 = RehearsalSlot(date(2024, 9, 5), 'thursday', 1900, 2100) # Should NOT conflict

conflicts_1 = check_for_conflict(rehearsal_1, parsed_constraints)
print(f"Checking Rehearsal 1 (Wednesday): Found conflicts: {conflicts_1}")

conflicts_2 = check_for_conflict(rehearsal_2, parsed_constraints)
print(f"Checking Rehearsal 2 (Thursday): Found conflicts: {conflicts_2}")

Checking Rehearsal 1 (Wednesday): Found conflicts: [(1800, 1930)]
Checking Rehearsal 2 (Thursday): Found conflicts: []


In [10]:
# Assuming all the classes (RehearsalSlot, constraints) and the transformer are defined above...

# --- Main Execution Logic ---
try:
    # 1. Instantiate the parser with the new grammar and transformer
    constraint_parser = Lark(constraint_grammar, start='start', parser='lalr', transformer=ConstraintTransformer())

    # 2. Define a complex unavailability string
    dancer_unavailability_string = "mo until 12, w 2-4, fri after 5pm, sunday"

    # 3. Parse the string
    parsed_constraints = constraint_parser.parse(dancer_unavailability_string)

    print(f"Input string: '{dancer_unavailability_string}'")
    print("Parsed constraint objects:")
    for c in parsed_constraints:
        print(f"  - {c}")
    print("\n")
    
    # 4. Define rehearsals to check
    rehearsal_ok = RehearsalSlot(date(2024, 9, 2), 'monday', 1300, 1500) # OK, it's after 1200
    rehearsal_conflict_1 = RehearsalSlot(date(2024, 9, 2), 'monday', 1100, 1230) # CONFLICT, overlaps with "until 1200"
    rehearsal_conflict_2 = RehearsalSlot(date(2024, 9, 4), 'wednesday', 1500, 1700) # CONFLICT, overlaps with 1400-1600
    rehearsal_conflict_3 = RehearsalSlot(date(2024, 9, 8), 'sunday', 1000, 1100) # CONFLICT, unavailable all day Sunday

    # 5. Check for conflicts
    print(f"Checking {rehearsal_ok.day_of_week} at {rehearsal_ok.start_time}:")
    print(f"  -> Conflicts: {check_for_conflict(rehearsal_ok, parsed_constraints)}\n")

    print(f"Checking {rehearsal_conflict_1.day_of_week} at {rehearsal_conflict_1.start_time}:")
    print(f"  -> Conflicts: {check_for_conflict(rehearsal_conflict_1, parsed_constraints)}\n")
    
    print(f"Checking {rehearsal_conflict_2.day_of_week} at {rehearsal_conflict_2.start_time}:")
    print(f"  -> Conflicts: {check_for_conflict(rehearsal_conflict_2, parsed_constraints)}\n")

    print(f"Checking {rehearsal_conflict_3.day_of_week} at {rehearsal_conflict_3.start_time}:")
    print(f"  -> Conflicts: {check_for_conflict(rehearsal_conflict_3, parsed_constraints)}\n")

    # 6. Test the validation
    print("Testing invalid input 'th after 25'...")
    invalid_string = "th after 25"
    constraint_parser.parse(invalid_string)

except Exception as e:
    print(f"Successfully caught expected error: {e}")

Input string: 'mo until 12, w 2-4, fri after 5pm, sunday'
Parsed constraint objects:
  - TimeOnDayConstraint(day_of_week='monday', start_time=0, end_time=1200)
  - TimeOnDayConstraint(day_of_week='wednesday', start_time=1400, end_time=1600)
  - TimeOnDayConstraint(day_of_week='friday', start_time=1700, end_time=2359)
  - DayOfWeekConstraint(day_of_week='sunday')


Checking monday at 1300:
  -> Conflicts: []

Checking monday at 1100:
  -> Conflicts: [(1100, 1200)]

Checking wednesday at 1500:
  -> Conflicts: [(1500, 1600)]

Checking sunday at 1000:
  -> Conflicts: [(1000, 1100)]

Testing invalid input 'th after 25'...
Successfully caught expected error: Invalid 24-hour format: Hour '25' cannot be greater than 24.


In [11]:
# Assuming 'constraint_parser' is your Lark instance with the ConstraintTransformer

# A string with a syntactic error ("zulu") and a semantic error ("th after 99")
test_text_multiple_errors = "Monday, zulu, th after 99, wed"

print(f"Checking: '{test_text_multiple_errors}'")
ok, constraints, errors = validate_and_parse_all_constraints(constraint_parser, test_text_multiple_errors)

if ok:
    print("\nSuccess! Parsed constraints:")
    for c in constraints:
        print(f"  - {c}")
else:
    print(f"\nFailure! Reasons:\n{errors}")

print("-" * 20)

Checking: 'Monday, zulu, th after 99, wed'

Failure! Reasons:
Found one or more invalid constraints:
- Constraint 'zulu' is not grammatically valid.
- Constraint 'th after 99' is not logically valid: Invalid 24-hour format: Hour '99' cannot be greater than 24.
--------------------


In [12]:
# A fully valid string for comparison
test_text_valid = "mo until 12, w 2-4, fri after 5pm"
print(f"Checking: '{test_text_valid}'")
ok, constraints, errors = validate_and_parse_all_constraints(constraint_parser, test_text_valid)

if ok:
    print("\nSuccess! Parsed constraints:")
    for c in constraints:
        print(f"  - {c}")
else:
    print(f"\nFailure! Reasons:\n{errors}")

Checking: 'mo until 12, w 2-4, fri after 5pm'

Success! Parsed constraints:
  - [TimeOnDayConstraint(day_of_week='monday', start_time=0, end_time=1200)]
  - [TimeOnDayConstraint(day_of_week='wednesday', start_time=1400, end_time=1600)]
  - [TimeOnDayConstraint(day_of_week='friday', start_time=1700, end_time=2359)]


In [13]:
# A string with a syntactic error ("zulu") and a semantic error ("th after 99")
test_text_multiple_errors = "Monday, zulu, th after 99, th after 5, wed"

print(f"Checking: '{test_text_multiple_errors}'")
ok, constraints, errors = validate_and_parse_all_constraints(constraint_parser, test_text_multiple_errors)

if ok:
    print("\nSuccess! Parsed constraints:")
    # Assuming your constraints have a __str__ or __repr__ method
    for c in constraints:
        print(f"  - {c}")
else:
    print(f"\nFailure! Reasons:\n{errors}")

Checking: 'Monday, zulu, th after 99, th after 5, wed'

Failure! Reasons:
Found one or more invalid constraints:
- Constraint 'zulu' is not grammatically valid.
- Constraint 'th after 99' is not logically valid: Invalid 24-hour format: Hour '99' cannot be greater than 24.


# Notes on EBNF

## The Jupyternaut Prescription for Lark Transformers

Here is a concise guide with minimal examples to serve as your mental model. Think of this as the "Standard Model" for connecting your grammar to your transformer.

The fundamental principle is: **For every possible path a rule can take, there must be a corresponding method in the transformer to handle it.**

### Case 1: The Default Rule (No Alias)

This is the simplest case. The method name in your transformer **must exactly match** the rule name in your grammar.

*   **Grammar Snippet (EBNF):**
    ```lark
    time_range: time "-" time
    ```
*   **How to Identify:** A simple rule name (`time_range`) defined with a colon (`:`).
*   **Transformer Snippet (Python):**
    ```python
    class MyTransformer(Transformer):
        # Method name matches the rule name
        def time_range(self, children):
            # children will be a list of the processed parts, e.g., [900, 1700]
            start_time, end_time = children
            return (start_time, end_time)
    ```

### Case 2: The Alias Rule (The `->` Arrow)

The alias (`->`) completely overrides the default behavior. Lark will ignore the rule name and look for a method matching the alias name.

*   **Grammar Snippet (EBNF):**
    ```lark
    time_range: time "-" time -> build_explicit_range
    ```
*   **How to Identify:** Look for the arrow `->` followed by a new name (`build_explicit_range`).
*   **Transformer Snippet (Python):**
    ```python
    class MyTransformer(Transformer):
        # Method name matches the ALIAS name
        def build_explicit_range(self, children):
            start_time, end_time = children
            return (start_time, end_time)
    ```

### Case 3: The "Or" Rule (Multiple Branches)

This is the one that caught you, and it's just a combination of the first two cases. You must treat each branch (separated by a `|`) as its own independent path.

*   **Grammar Snippet (EBNF):**
    ```lark
    range: time "-" time -> build_explicit_range  // Path 1 (Alias)
         | "until"i time                        // Path 2 (Default)
    ```
*   **How to Identify:** Look for the vertical bar `|` separating different patterns. Check each pattern for an alias.
*   **Transformer Snippet (Python):** You need one method for each path.
    ```python
    class MyTransformer(Transformer):
        # Method for Path 1, matching the alias
        def build_explicit_range(self, children):
            start_time, end_time = children
            return (start_time, end_time)

        # Method for Path 2, matching the RULE name ("range")
        def range(self, children):
            # This branch only has one child: the processed `time`
            end_time = children[0]
            return (0, end_time) # from start of day until the time
    ```

### Case 4: Transforming Terminals (The Uppercase Tokens)

You can also write methods for the "atoms" of your grammar—the Terminals (usually in `ALL_CAPS`). This is extremely useful for converting raw text into useful data types like numbers or dates.

*   **Grammar Snippet (EBNF):**
    ```lark
    INT: /[0-9]+/
    ```
*   **How to Identify:** A token name in all caps.
*   **Transformer Snippet (Python):**
    ```python
    class MyTransformer(Transformer):
        # Method name matches the Terminal name
        def INT(self, number_token):
            # The argument is the token object itself
            # We convert its string value to a Python integer
            return int(number_token.value)
    ```

The Jupyternaut Prescription for Lark Transformers¶
Here is a concise guide with minimal examples to serve as your mental model. Think of this as the "Standard Model" for connecting your grammar to your transformer.

The fundamental principle is: For every possible path a rule can take, there must be a corresponding method in the transformer to handle it.

Case 1: The Default Rule (No Alias)
This is the simplest case. The method name in your transformer must exactly match the rule name in your grammar.

Grammar Snippet (EBNF):
time_range: time "-" time
How to Identify: A simple rule name (time_range) defined with a colon (:).
Transformer Snippet (Python):
class MyTransformer(Transformer):
    # Method name matches the rule name
    def time_range(self, children):
        # children will be a list of the processed parts, e.g., [900, 1700]
        start_time, end_time = children
        return (start_time, end_time)
Case 2: The Alias Rule (The -> Arrow)
The alias (->) completely overrides the default behavior. Lark will ignore the rule name and look for a method matching the alias name.

Grammar Snippet (EBNF):
time_range: time "-" time -> build_explicit_range
How to Identify: Look for the arrow -> followed by a new name (build_explicit_range).
Transformer Snippet (Python):
class MyTransformer(Transformer):
    # Method name matches the ALIAS name
    def build_explicit_range(self, children):
        start_time, end_time = children
        return (start_time, end_time)
Case 3: The "Or" Rule (Multiple Branches)
This is the one that caught you, and it's just a combination of the first two cases. You must treat each branch (separated by a |) as its own independent path.

Grammar Snippet (EBNF):
range: time "-" time -> build_explicit_range  // Path 1 (Alias)
     | "until"i time                        // Path 2 (Default)
How to Identify: Look for the vertical bar | separating different patterns. Check each pattern for an alias.
Transformer Snippet (Python): You need one method for each path.
class MyTransformer(Transformer):
    # Method for Path 1, matching the alias
    def build_explicit_range(self, children):
        start_time, end_time = children
        return (start_time, end_time)

    # Method for Path 2, matching the RULE name ("range")
    def range(self, children):
        # This branch only has one child: the processed `time`
        end_time = children[0]
        return (0, end_time) # from start of day until the time
Case 4: Transforming Terminals (The Uppercase Tokens)
You can also write methods for the "atoms" of your grammar—the Terminals (usually in ALL_CAPS). This is extremely useful for converting raw text into useful data types like numbers or dates.

Grammar Snippet (EBNF):
INT: /[0-9]+/
How to Identify: A token name in all caps.
Transformer Snippet (Python):
class MyTransformer(Transformer):
    # Method name matches the Terminal name
    def INT(self, number_token):
        # The argument is the token object itself
        # We convert its string value to a Python integer
        return int(number_token.value)
This should be a great starting point for your personal notes. Feel free to expand on it with your own discoveries, and don't hesitate to ask if any other concepts come up! I'm always here to help.
